Grab relevant metrics from github.

In [ ]:
!pip install -q --pre github3.py

In [ ]:
import json
import os

with open("kernels/kernels.json") as f:
    kernels = json.load(f)

In [ ]:
import github3 as gh3
gh = gh3.login(os.environ["GITHUB_USER"], token=os.environ["GITHUB_TOKEN"])

In [ ]:
repos = []
for key, kernel in kernels["kernels"].items():
    if "actions" in kernel and "act:github" in kernel["actions"]:
        owner, name = kernel["actions"]["act:github"].split(":")[1].split("/")
    elif "github.com" in kernel["url"]:
        owner, name = (kernel["url"]
                       .replace("https://github.com/", "")
                       .split("/")
                       )[0:2]
        kernel.setdefault("actions", {})["act:github"] = "gh:{}{}".format(owner, name)
    else:
        continue

    repo = gh.repository(owner, name)
    if isinstance(repo, gh3.repos.repo.Repository):
        repos.append([kernel, repo])

Update metrics

In [ ]:
for kernel, repo in repos:
    kernel.setdefault("metrics", {}).update({
        "gh:stargazers":  repo.stargazers_count,
        "gh:forks":  repo.fork_count,
        "gh:updated": repo.updated_at.isoformat()
    })

Find logo

In [ ]:
import time
for kernel, repo in repos:
    if "logo" in kernel:
        continue
    time.sleep(1)
    for image in gh.search_code("repo:{} extension:png".format(repo.full_name)):
        if image.name == "logo-64x64.png":
            kernel["logo"] = (image.html_url
                .replace("github.com", "cdn.rawgit.com")
                .replace("/blob/", "/")
            )

In [ ]:
import time
for kernel, repo in repos:
    if "actions" in kernel and "act:view" in kernel["actions"]:
        continue
    time.sleep(2)
    for nb in gh.search_code("repo:{} extension:ipynb".format(repo.full_name)):
        kernel["actions"]["act:view"] = (
            "http://nbviewer.jupyter.org/github/{}/tree/master/{}".format(
                repo.full_name,
                os.path.dirname(nb.path)
            )
        )

In [ ]:
for kernel, repo in repos:
    if "description" not in kernel and repo.description:
        kernel["description"] = repo.description

In [ ]:
with open("kernels/kernels.json", "w") as f:
    f.write(json.dumps(kernels, indent=2, sort_keys=True))